# Grid Search

<span>Beware full grid search can be more computation intensive depending on your parameters. We use a K-Nearest Neighbours model for its example. After the Full Grid Search is done, you can pull the best parameter for your model, and as well as take a look a the history of the previous combination of parameters.</span>
    
### Import Preliminaries

In [16]:
# Import modules
import numpy as np
import pandas as pd

from sklearn.cross_validation import cross_val_score
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import warnings


# Set warning options
warnings.filterwarnings('ignore');

### Import Data

In [17]:
# Loading iris data set
iris = load_iris()
X, y = iris.data, iris.target

### Set up Model and Grid Search

In [18]:
# Initialize the k-nearest neigbours model
knn_model = KNeighborsClassifier(n_neighbors=5, weights='uniform', 
                                 metric ='minkowski', p=2)

# Setting up the grid
grid = {'n_neighbors':list(range(5,10)), 
        'weights':['uniform', 'distance'],
        'p':[1,2,3,4], }

# Initialize with GridSearchCV with grid
grid_search = GridSearchCV(estimator=knn_model, param_grid=grid, 
                     scoring='accuracy', n_jobs=1, refit=True, cv=10,
                     return_train_score=True)

# Fit moedl
grid_search.fit(X,y)

GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [5, 6, 7, 8, 9], 'weights': ['uniform', 'distance'], 'p': [1, 2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

### Grid Search Score

In [19]:
# Print the best grid search score
print('Accuracy of best parameters: %.3f'%grid_search.best_score_)
print('Best parameters: %s' %grid_search.best_params_)

Accuracy of best parameters: 0.980
Best parameters: {'n_neighbors': 6, 'p': 3, 'weights': 'uniform'}


### Baseline Cross Validation Score


In [20]:
# print the baseline score of our model with default parameters
scores = cross_val_score(knn_model, X, y, cv=10, scoring='accuracy', n_jobs=1)
print ('Baesline with default parameters: %.3f' %np.mean(scores))

Baesline with default parameters: 0.967


### Viewing Grid Scores

In [21]:
# View the raw grid search scores
grid_search.grid_scores_

[mean: 0.96667, std: 0.04472, params: {'n_neighbors': 5, 'p': 1, 'weights': 'uniform'},
 mean: 0.96667, std: 0.04472, params: {'n_neighbors': 5, 'p': 1, 'weights': 'distance'},
 mean: 0.96667, std: 0.04472, params: {'n_neighbors': 5, 'p': 2, 'weights': 'uniform'},
 mean: 0.96667, std: 0.04472, params: {'n_neighbors': 5, 'p': 2, 'weights': 'distance'},
 mean: 0.96667, std: 0.04472, params: {'n_neighbors': 5, 'p': 3, 'weights': 'uniform'},
 mean: 0.96667, std: 0.04472, params: {'n_neighbors': 5, 'p': 3, 'weights': 'distance'},
 mean: 0.96667, std: 0.04472, params: {'n_neighbors': 5, 'p': 4, 'weights': 'uniform'},
 mean: 0.96667, std: 0.04472, params: {'n_neighbors': 5, 'p': 4, 'weights': 'distance'},
 mean: 0.94000, std: 0.05538, params: {'n_neighbors': 6, 'p': 1, 'weights': 'uniform'},
 mean: 0.95333, std: 0.05207, params: {'n_neighbors': 6, 'p': 1, 'weights': 'distance'},
 mean: 0.96667, std: 0.04472, params: {'n_neighbors': 6, 'p': 2, 'weights': 'uniform'},
 mean: 0.96667, std: 0.0447

In [22]:
results = pd.DataFrame(grid_search.cv_results_)
results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,0.000326,0.000085,0.000524,0.000080,5,1,uniform,"{'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}",1.0,0.933333,...,0.962963,0.970370,0.985185,0.955556,0.977778,0.955556,0.962963,0.962963,0.966667,0.008920
1,0.000262,0.000013,0.000477,0.000033,5,1,distance,"{'n_neighbors': 5, 'p': 1, 'weights': 'distance'}",1.0,0.933333,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
2,0.000254,0.000012,0.000429,0.000007,5,2,uniform,"{'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}",1.0,0.933333,...,0.962963,0.970370,0.985185,0.962963,0.977778,0.962963,0.962963,0.970370,0.968889,0.007258
3,0.000245,0.000001,0.000457,0.000019,5,2,distance,"{'n_neighbors': 5, 'p': 2, 'weights': 'distance'}",1.0,0.933333,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
4,0.000266,0.000018,0.000520,0.000012,5,3,uniform,"{'n_neighbors': 5, 'p': 3, 'weights': 'uniform'}",1.0,0.933333,...,0.962963,0.970370,0.985185,0.962963,0.977778,0.962963,0.962963,0.970370,0.968889,0.007258
5,0.000299,0.000049,0.000565,0.000044,5,3,distance,"{'n_neighbors': 5, 'p': 3, 'weights': 'distance'}",1.0,0.933333,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
6,0.000326,0.000095,0.000600,0.000098,5,4,uniform,"{'n_neighbors': 5, 'p': 4, 'weights': 'uniform'}",1.0,0.933333,...,0.962963,0.970370,0.985185,0.970370,0.977778,0.962963,0.962963,0.970370,0.969630,0.006988
7,0.000298,0.000033,0.000571,0.000044,5,4,distance,"{'n_neighbors': 5, 'p': 4, 'weights': 'distance'}",1.0,0.933333,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
8,0.000284,0.000033,0.000479,0.000054,6,1,uniform,"{'n_neighbors': 6, 'p': 1, 'weights': 'uniform'}",1.0,0.933333,...,0.955556,0.955556,0.977778,0.948148,0.970370,0.962963,0.948148,0.962963,0.959259,0.008920
9,0.000295,0.000047,0.000548,0.000101,6,1,distance,"{'n_neighbors': 6, 'p': 1, 'weights': 'distance'}",1.0,0.933333,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


Author: Kavi Sekhon